In [1]:
# https://towardsdatascience.com/how-to-gain-state-of-the-art-result-on-tabular-data-with-deep-learning-and-embedding-layers-d1eb6b83c52c

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import unidecode
import re
import random
from collections import Counter

In [4]:
random.seed(34)

In [5]:
from exp.nb_01 import load_data
from exp.nb_02 import gini_score

In [6]:
def write_down_features(df, label_df):
    train_df = df[0:n_train]
    test_df = df[n_train:]
    train_df, test_df = train_df.reset_index(drop=True), test_df.reset_index(drop=True)
    train_df['label'] = list(label_df)
    train_df.to_pickle('data/train_feature.pickle')
    test_df.to_pickle('data/test_feature.pickle')

In [7]:
#export
# n_valid = 4000

In [8]:
train_df, test_df = load_data(keep_df=True)

/home/vule/miniconda3/envs/thanos/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/home/vule/miniconda3/envs/thanos/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [9]:
source_df = pd.concat([train_df, test_df])

/home/vule/miniconda3/envs/thanos/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
# train_df['label'] = train_df.label.apply(lambda x: 0.01 if x == 0 else 0.99)

In [11]:
len(train_df), len(train_df.drop_duplicates())

(30000, 30000)

In [12]:
cat_features = ['province', 'district', 'maCv',
                'FIELD_7', 'FIELD_8', 'FIELD_9',
                'FIELD_10', 'FIELD_12', 'FIELD_13', 'FIELD_17', 
                'FIELD_24', 'FIELD_35', 'FIELD_39', 'FIELD_40',
                'FIELD_41', 'FIELD_42', 'FIELD_43', 
                'FIELD_44', 'FIELD_45']
bool_features = ['FIELD_1', 'FIELD_2', 'FIELD_14', 'FIELD_15', 'FIELD_18', 'FIELD_19', 
                 'FIELD_20', 'FIELD_23', 'FIELD_25', 
                 'FIELD_26', 'FIELD_27', 'FIELD_28', 
                 'FIELD_29', 'FIELD_30', 'FIELD_31', 'FIELD_32', 'FIELD_33','FIELD_34',
                 'FIELD_36', 'FIELD_37', 'FIELD_38', 
                 'FIELD_46', 'FIELD_47', 'FIELD_48', 'FIELD_49']
num_features = [col for col in test_df.columns if col not in cat_features+bool_features]
num_features.remove('id')

print(len(cat_features), len(bool_features), len(num_features))

19 25 18


In [13]:
addition_cat_features = []
alphabe = list('abcdefghtkmn')
for c in num_features:
    len(c)
    keys = list(set(source_df[source_df[c].notna()][c]))
    values = alphabe[:len(keys)]
    mapper = dict(zip(keys, values))
    if len(set(train_df[train_df[c].notna()][c])) < 10:
#         print(c, set(train_df[train_df[c].notna()][c]), mapper)
        addition_cat_features += [f"{c}_cat"]
        train_df[f"{c}_cat"] = train_df[c].apply(lambda x: mapper[x] if x in mapper else x)
        test_df[f"{c}_cat"] = test_df[c].apply(lambda x: mapper[x] if x in mapper else x)
cat_features += addition_cat_features

In [14]:
# update sorce df
source_df = pd.concat([train_df, test_df])

/home/vule/miniconda3/envs/thanos/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [15]:
train_df[num_features].sample(15)

,age_source1,age_source2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,FIELD_11,FIELD_16,FIELD_21,FIELD_22,FIELD_50,FIELD_51,FIELD_52,FIELD_53,FIELD_54,FIELD_55,FIELD_56,FIELD_57
16711,23.0,23.0,714.0,0.0,4.0,0.0,0,1.0,1.0,11.0,29.770,54.261,30.955,31.171,0.00,0.16,0.0,0.0
9391,NaN,26.0,718.0,0.0,0.0,0.0,12,1.0,1.0,0.0,60.946,4.413,30.955,31.171,0.12,0.00,0.0,0.0
3536,NaN,30.0,3649.0,0.0,0.0,0.0,0,1.0,1.0,0.0,29.770,36.851,30.955,31.171,0.00,0.12,0.0,0.0
27894,36.0,36.0,2168.0,0.0,1.0,0.0,0,1.0,1.0,3.0,29.770,43.858,30.955,31.171,0.00,0.14,0.0,0.0
8482,49.0,49.0,2174.0,0.0,5.0,0.0,5,1.0,1.0,66.0,29.770,30.404,30.955,31.171,0.00,0.09,0.0,0.0
19503,57.0,57.0,-1.0,0.0,7.0,0.0,12,1.0,1.0,12.0,29.770,12.674,30.955,31.171,0.00,0.03,0.0,0.0
21198,NaN,NaN,-1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8755,45.0,45.0,-1.0,0.0,3.0,1.0,12,1.0,1.0,41.0,29.770,13.789,30.955,31.171,0.00,0.03,0.0,0.0
6017,NaN,NaN,724.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24969,23.0,23.0,-1.0,0.0,6.0,0.0,12,1.0,1.0,38.0,29.770,35.535,30.955,31.171,0.00,0.12,0.0,0.0


In [16]:
positive_df = train_df[train_df.label > 0.5]

In [17]:
def filling_na_bool_features(train_df, df, keep=0.15):
    for c in bool_features:
        population = list(train_df[train_df[c].notna()][c])
        df[c] = df[c].apply(lambda x: random.choice(population) if pd.isna(x) and random.random() < keep else x)
    df['version'] = 'bool_features'
    return df

def filling_na_cat_features(train_df, df, keep=0.15):
    for c in cat_features:
        population = list(train_df[train_df[c].notna()][c])
        df[c] = df[c].apply(lambda x: random.choice(population) if pd.isna(x) and random.random() < keep else x)
    df['version'] = 'cat_features'
    return df

def filling_na_num_features(train_df, df, keep=0.15):
    for c in num_features:
        population = list(train_df[train_df[c].notna()][c])
        df[c] = df[c].apply(lambda x: random.choice(population) if pd.isna(x) and random.random() < keep else x)
    df['version'] = 'num_features'
    return df

In [18]:
def sample_neighbor(population, x):
    population.sort()
    n_population = len(population)
    delta = int(n_population * 0.1 / 2)
    if x not in population:
        return x
    index = population.index(x)
    l, r = max(0, index - delta), min(n_population, index + delta)
    return random.choice(population[l:r])

def shiff_num_features(train_df, df, keep=0.15):
    for c in num_features:
        population = list(set(train_df[train_df[c].notna()][c]))
        if len(population) > 20:
            df[c] = df[c].apply(lambda x: sample_neighbor(population, x) if (not pd.isna(x)) and (x != -1) and (random.random() < keep) else x)
    return df

In [19]:
train_df[num_features]

,age_source1,age_source2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,FIELD_11,FIELD_16,FIELD_21,FIELD_22,FIELD_50,FIELD_51,FIELD_52,FIELD_53,FIELD_54,FIELD_55,FIELD_56,FIELD_57
0,NaN,NaN,2547.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44.0,44.0,-1.0,1.0,1.0,1.0,12,2.0,1.0,4.0,29.77,17.561,30.955,31.171,0.0,0.04,0.0,0.0
2,30.0,30.0,3273.0,1.0,4.0,0.0,12,1.0,1.0,54.0,29.77,56.512,30.955,31.171,0.0,0.16,0.0,0.0
3,43.0,NaN,3991.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21.0,21.0,1450.0,0.0,2.0,0.0,12,1.0,1.0,24.0,29.77,13.789,30.955,31.171,0.0,0.03,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,28.0,28.0,720.0,2.0,6.0,0.0,0,1.0,1.0,36.0,29.77,33.503,30.955,31.171,0.0,0.11,0.0,0.0
29996,34.0,34.0,3627.0,0.0,8.0,2.0,6,1.0,1.0,104.0,29.77,41.194,30.955,31.171,0.0,0.13,0.0,0.0
29997,29.0,NaN,3651.0,2.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,39.0,39.0,-1.0,2.0,2.0,0.0,12,1.0,1.0,24.0,29.77,21.478,30.955,31.171,0.0,0.05,0.0,0.0


In [20]:
len(num_features)

18

In [21]:
def filling_na_3_features(train_df, positive_df):
    positive_df['version'] = '1'
    v1_positive = pd.concat([filling_na_bool_features(train_df, positive_df.copy(), 0.25) for _ in range(2)])
    v1_positive = pd.concat([positive_df, v1_positive]).drop_duplicates(['id'] + bool_features)
    v1_positive = v1_positive[v1_positive.version == 'bool_features']

    v2_positive = pd.concat([filling_na_cat_features(train_df, v1_positive.copy(), 0.15) for _ in range(1)])
    v2_positive = pd.concat([v1_positive, v2_positive]).drop_duplicates(['id'] + cat_features)
    v2_positive = v2_positive[v2_positive.version == 'cat_features']

    v3_positive = pd.concat([filling_na_num_features(train_df, v2_positive.copy(), 0.15) for _ in range(1)])
    v3_positive = pd.concat([v2_positive, v3_positive]).drop_duplicates(['id'] + num_features)
    v3_positive = v3_positive[v3_positive.version == 'num_features']
    del v3_positive['version']

    print(len(positive_df), len(v1_positive), len(v2_positive), len(v3_positive))
    return v3_positive


def augmente_data_v1(train_df, positive_df, w=[2,2,2]):
    # just random do filling_na features
    v1_positive = pd.concat([filling_na_bool_features(train_df, positive_df.copy(), 0.25) for _ in range(w[0])])
    v2_positive = pd.concat([filling_na_cat_features(train_df, v1_positive.copy(), 0.15) for _ in range(w[1])])
    v3_positive = pd.concat([filling_na_num_features(train_df, v2_positive.copy(), 0.15) for _ in range(w[2])])
    del v1_positive['version']
    del v2_positive['version']
    del v3_positive['version']
    print(len(positive_df), 
          len(pd.concat([positive_df, v1_positive]).drop_duplicates()), 
          len(pd.concat([positive_df, v2_positive]).drop_duplicates()), 
          len(pd.concat([positive_df, v3_positive]).drop_duplicates()), )
    return v3_positive

def augmente_data_v2(train_df, positive_df, w=[2,2,2]):
    # just random do filling_na only 1 columns each row.
    v1_positive = pd.concat([filling_na_bool_features(train_df, positive_df.copy(), 0.25) for _ in range(w[0])])
    v2_positive = pd.concat([filling_na_cat_features(train_df, positive_df.copy(), 0.15) for _ in range(w[1])])
    v3_positive = pd.concat([filling_na_num_features(train_df, positive_df.copy(), 0.15) for _ in range(w[2])])
    del v1_positive['version']
    del v2_positive['version']
    del v3_positive['version']
    print(len(positive_df), 
          len(pd.concat([positive_df, v1_positive]).drop_duplicates()), 
          len(pd.concat([positive_df, v2_positive]).drop_duplicates()), 
          len(pd.concat([positive_df, v3_positive]).drop_duplicates()), )
    return pd.concat([v1_positive, v2_positive, v3_positive])

def augmente_data_v3(train_df, positive_df, w=[1,1,1]):
    # just random do filling_na only 1 columns each row. and ship num values
    agumented_data = augmente_data_v2(train_df, positive_df, w)
    print(len(agumented_data))
    agumented_data = shiff_num_features(train_df, agumented_data, keep=0.3)
    print(len(positive_df), 
          len(pd.concat([positive_df, agumented_data]).drop_duplicates()))
    return agumented_data

agumented_data_v1 = augmente_data_v1(source_df, train_df, [1,1,1])
agumented_pos_v2 = augmente_data_v2(source_df, positive_df, [4,4,4])
agumented_data_v3 = augmente_data_v3(source_df, train_df, [1,1,1])
agumented_data = pd.concat([agumented_data_v1, agumented_pos_v2, agumented_data_v3])

30000 39507 40998 41480
486 1284 1350 1263
30000 39489 40681 39414
90000
30000 116075


In [22]:
len(positive_df), len(agumented_data)

(486, 125832)

In [23]:
len(pd.concat([train_df, agumented_data]).drop_duplicates())

129990

In [24]:
assert(len(train_df.columns) - 1 == len(test_df.columns))

In [25]:
train_df = pd.concat([train_df, agumented_data]).drop_duplicates().sort_values('id')

In [26]:
len(train_df[train_df['id'] > 24000])

25891

In [27]:
agumented_test = augmente_data_v3(source_df, test_df, [1,1,1])

20000 26405 27201 26348
60000
20000 77392


In [28]:
test_df = pd.concat([test_df, agumented_test]).drop_duplicates().sort_values('id')

In [29]:
#export
n_valid = 25891

In [30]:
n_train, n_test = len(train_df), len(test_df)
n_train, n_test

(129990, 77392)

In [31]:
train_df.columns

Index(['id', 'label', 'province', 'district', 'age_source1', 'age_source2',
       'maCv', 'FIELD_1', 'FIELD_2', 'FIELD_3', 'FIELD_4', 'FIELD_5',
       'FIELD_6', 'FIELD_7', 'FIELD_8', 'FIELD_9', 'FIELD_10', 'FIELD_11',
       'FIELD_12', 'FIELD_13', 'FIELD_14', 'FIELD_15', 'FIELD_16', 'FIELD_17',
       'FIELD_18', 'FIELD_19', 'FIELD_20', 'FIELD_21', 'FIELD_22', 'FIELD_23',
       'FIELD_24', 'FIELD_25', 'FIELD_26', 'FIELD_27', 'FIELD_28', 'FIELD_29',
       'FIELD_30', 'FIELD_31', 'FIELD_32', 'FIELD_33', 'FIELD_34', 'FIELD_35',
       'FIELD_36', 'FIELD_37', 'FIELD_38', 'FIELD_39', 'FIELD_40', 'FIELD_41',
       'FIELD_42', 'FIELD_43', 'FIELD_44', 'FIELD_45', 'FIELD_46', 'FIELD_47',
       'FIELD_48', 'FIELD_49', 'FIELD_50', 'FIELD_51', 'FIELD_52', 'FIELD_53',
       'FIELD_54', 'FIELD_55', 'FIELD_56', 'FIELD_57', 'FIELD_6_cat',
       'FIELD_16_cat', 'FIELD_21_cat', 'FIELD_50_cat', 'FIELD_54_cat'],
      dtype='object')

In [32]:
train_df[train_df.label == 1].sample(10)[['province', 'district', 'maCv', 'FIELD_7', 'FIELD_8', 'FIELD_9', 'FIELD_10', 'FIELD_11', 'FIELD_12', 'FIELD_13', 'FIELD_17', 'FIELD_18', 'FIELD_19', 'FIELD_20', 'FIELD_23', 'FIELD_24', 'FIELD_25', 'FIELD_26', 'FIELD_27', 'FIELD_28', 'FIELD_29', 'FIELD_30', 'FIELD_31', 'FIELD_35', 'FIELD_36', 'FIELD_37', 'FIELD_38', 'FIELD_39', 'FIELD_40', 'FIELD_41', 'FIELD_42', 'FIELD_43', 'FIELD_44', 'FIELD_45', 'FIELD_47', 'FIELD_48', 'FIELD_49']]

,province,district,maCv,FIELD_7,FIELD_8,FIELD_9,FIELD_10,FIELD_11,FIELD_12,FIELD_13,...,FIELD_39,FIELD_40,FIELD_41,FIELD_42,FIELD_43,FIELD_44,FIELD_45,FIELD_47,FIELD_48,FIELD_49
15284,NaN,NaN,NaN,[],NaN,na,GH,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,Two,NaN,False,False,True
25201,NaN,Huyện Thanh Trì,Công Nhân,[],FEMALE,DN,GH,6,0,YN,...,None,None,I,Zezo,None,Two,2,False,True,False
19702,Tỉnh Bình Phước,Huyện Hớn Quản,NaN,"['TE', 'GD', 'TE', 'TE', 'TE']",NaN,na,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True
2386,NaN,NaN,NaN,[],NaN,na,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True
5879,Tỉnh Đắk Nông,Thị xã Gia Nghĩa,None,"['DN', 'DN', 'DN', 'DN']",FEMALE,DN,GH,12,0,TG,...,VN,None,I,Zezo,None,One,1,True,True,True
21015,NaN,NaN,None,NaN,NaN,na,NaN,NaN,NaN,NaN,...,VN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True
4515,NaN,NaN,NaN,[],NaN,na,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
26253,NaN,NaN,None,[],FEMALE,GD,T1,12,0,BI,...,VN,None,V,Zezo,None,One,1,False,False,True
21882,NaN,NaN,NaN,[],NaN,na,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True
21013,NaN,NaN,NaN,[],NaN,na,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True


In [33]:
label_df = train_df['label']
del train_df['label']

In [34]:
df = pd.concat([train_df, test_df]); len(df)

207382

In [35]:
original_columns = list(df.columns)
original_columns.remove('id')

### Lower and unicode all columns

In [36]:
for c in original_columns:
    not_na_id = df[c].notna()
    try:
        df.loc[not_na_id, c] = df[not_na_id][c].str.lower().apply(lambda x: re.sub(' +', ' ', unidecode.unidecode(x)))
    except BaseException as e:
        pass

In [37]:
len(df)

207382

## Categorical & Boolean Features

#### 1. Missing and None values

#### Count isna and none columns

In [38]:
### count isna columns
for c in original_columns:
    df[f"{c}_isna"] = df[c].isna()
    
df['n_cat_na'] = df[cat_features].isnull().sum(axis=1)
df['n_boolean_na'] = df[bool_features].isnull().sum(axis=1)

In [39]:
### count none columns
df['n_cat_none'] = df[cat_features].apply(lambda xs: len([x for x in xs if x == 'none' or x == 'na']), axis=1)
df['n_boolean_none'] = df[bool_features].apply(lambda xs: len([x for x in xs if x == 'none' or x == 'na']), axis=1)
for c in original_columns:
    df[f"{c}_isnone"] = df[c] == 'none'

/home/vule/miniconda3/envs/thanos/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [40]:
df.sample(3)

,id,province,district,age_source1,age_source2,maCv,FIELD_1,FIELD_2,FIELD_3,FIELD_4,...,FIELD_53_isnone,FIELD_54_isnone,FIELD_55_isnone,FIELD_56_isnone,FIELD_57_isnone,FIELD_6_cat_isnone,FIELD_16_cat_isnone,FIELD_21_cat_isnone,FIELD_50_cat_isnone,FIELD_54_cat_isnone
12557,42557,tinh thai binh,huyen thai thuy,20.0,22.0,none,1,1.0,713.0,0.0,...,False,False,False,False,False,False,False,False,False,False
16516,16516,tinh binh duong,NaN,NaN,20.0,none,1,1.0,346.0,0.0,...,False,False,False,False,False,False,False,False,False,False
19804,19804,tinh bac lieu,huyen hong dan,23.0,23.0,none,1,1.0,2167.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [41]:
assert len(df) == n_train + n_test

#### 2.Ordinal categorical features

In [42]:
for c in ['FIELD_35', 'FIELD_41', 'FIELD_42', 'FIELD_43', 'FIELD_44']:
    print(c, set(df[c]))
ordinal_map = {'zezo': 0, '0': 0, 'zero': 0,
               'one': 1, 'i': 1, 'a': 1,
               'two': 2, 'ii': 2, 'b': 2,
               'three': 3, 'iii': 3, 'c': 3,
               'four': 4, 'iv': 4, 'd': 4,
               '5': 5, 'v': 5}
ordinal_features = []
for c in ['FIELD_35', 'FIELD_41', 'FIELD_42', 'FIELD_43', 'FIELD_44']:
    ordinal_features.append(f"{c}_Ordinal")
#     df[f"{c}_Ordinal"] = df[c].apply(lambda x: ordinal_map[x] if x in ordinal_map else x)

FIELD_35 {nan, 'two', 'four', 'zero', 'one', 'three'}
FIELD_41 {nan, 'iii', 'i', 'none', 'ii', 'iv', 'v'}
FIELD_42 {nan, 'one', 'zezo', 'none'}
FIELD_43 {nan, '0', 'd', 'c', 'a', '5', 'b', 'none'}
FIELD_44 {nan, 'one', 'none', 'two'}


#### 3.Ordinal categorical features

In [43]:
df.loc[:, cat_features] = df[cat_features].fillna('missing')

#### 4. Categorical features với nhiều hơn 10 unique values - Count Encoding

In [44]:
from collections import Counter
from exp.nb_01 import cv_mapper

not_na_id = df.maCv.notna()
for k, v in cv_mapper.items():
    df.loc[not_na_id, 'maCv'] = df[not_na_id].maCv.apply(lambda x: v if k in x else x)

In [45]:
for c in cat_features:
    if c != 'FIELD_7':
        temp = df.groupby(c).size().reset_index()
        temp.columns = [c, f"{c}_CE"]
        df = df.merge(temp)

In [46]:
df.sort_values('id', inplace=True)

In [47]:
assert len(df) == n_train + n_test

### Province, District

In [48]:
df['is_city'] = df.province.str.contains('thanh pho') | df.district.str.contains('thanh pho')

### Age_source 1 2

In [49]:
df['age1_less_18'] = df['age_source1'] < 18
df['age2_less_18'] = df['age_source2'] < 18

df['age1_gt_65'] = df['age_source1'] > 65
df['age2_gt_65'] = df['age_source2'] > 65
df['age_mean'] = (df['age_source1'] + df['age_source2']) / 2

### Numerical Features

In [50]:
for c in num_features:
    df[c] = df[c].apply(lambda x: -1 if x == 'none' else x)

#### Impute missing values

In [51]:
for c in num_features:
    try:
        df[c] = df[c].astype('float64')
        df[f"{c}_IM_const"] = df[c].fillna(-1)
        df[f"{c}_IM_mean"] = df[c].fillna(df[c].mean())
        df[f"{c}_IM_median"] = df[c].fillna(df[c].median())
    except:
        print(c)

### FIELD

### FIELD_7

In [52]:
df['FIELD_7'] = df[['FIELD_7']].fillna('[]')

In [53]:
df['FIELD_7'] = df['FIELD_7'].apply(lambda x: x.replace("'", '').replace('[', '').replace(']', '').split(', '))
df['FIELD_7'] = df['FIELD_7'].apply(lambda xs: [x for x in xs if x != ''])

In [54]:
df['FIELD_9_IN_7'] = df[['FIELD_7', 'FIELD_9']].apply(lambda xs: xs[1] in xs[0], axis=1)

In [55]:
df['len_f7'] = df['FIELD_7'].apply(lambda x: len(x))

In [56]:
value_f7 = set([x for xs in df['FIELD_7'] for x in xs])

In [57]:
df['FIELD_7_temp'] = df['FIELD_7'].apply(lambda x: Counter(x))

In [58]:
for v in value_f7:
    df[f"f7_{v}"] = df['FIELD_7_temp'].apply(lambda c: c[v])

In [59]:
for v in value_f7:
    df[f"f7_{v}_boolean"] = df['FIELD_7'].apply(lambda x: v in x)

In [60]:
del df['FIELD_7']
del df['FIELD_7_temp']

In [61]:
len(df)

207382

In [62]:
len(df.drop_duplicates())

207382

In [63]:
write_down_features(df, label_df)

In [64]:
!python notebook2script.py 05_enrich_feature.ipynb

Converted 05_enrich_feature.ipynb to exp/nb_05.py
